In [1]:
from TwitterAPI import TwitterAPI
import pandas as pd
from tqdm import tqdm
import os

In [2]:
keys = dict(consumer_key=os.environ['TWITTER_KEY'],
            consumer_secret=os.environ['TWITTER_SECRET'],
            access_token_key=os.environ['TWITTER_ACCESS_TOKEN_KEY'],
            access_token_secret=os.environ['TWITTER_ACCESS_TOKEN_SECRET'])
api = TwitterAPI(**keys)

In [6]:
# Load data
info_with_twitter = pd.read_csv('../data/congressperson_all_info.csv',
                                index_col=0)

# Initialize params
slug = "congress"
handle = "choldgraf"

In [7]:
# Find all of my lists
resp = api.request('lists/ownerships', params=dict(screen_name=handle,
                                                   count=1000))
lists_congresspeople = [ii for ii in resp.json()['lists']
                        if slug + '-' in ii['name']]
print('Destroying: {} lists'.format(len(lists_congresspeople)))

# Delete the ones created by this script
for i_list in tqdm(lists_congresspeople):
    i_id = i_list['id']
    resp = api.request('lists/destroy', {'list_id': i_id})

  0%|          | 0/50 [00:00<?, ?it/s]

Destroying: 50 lists


100%|██████████| 50/50 [00:15<00:00,  1.47it/s]


In [8]:
# Create a list for each state
ids = []
for state, values in tqdm(list(info_with_twitter.groupby('state'))):
    if state is None:
        continue

    # Create the list
    value = '{}-{}'.format(slug, state.replace(' ', '_'))
    desc = ("A collection of twitter accounts for congresspeople "
            "in the state of {}".format(state))
    resp = api.request('lists/create', params={'name': value, 'description':desc})
    this_id = resp.json()['id']
    this_url = resp.json()['uri']
    if resp.status_code != 200:
        print('Failed to create list')
        break
    ids.append((state, this_id, this_url))

columns = ['State', 'ID', 'URL']
df_ids = pd.DataFrame(ids, columns=columns)
df_ids['URL'] = df_ids['URL'].map(lambda a: 'twitter.com' + a)
df_ids['Link'] = df_ids['URL'].map(lambda a: '<a href="http://{}">link</a>'.format(a))
df_ids = df_ids[['State', 'Link', 'URL', 'ID']]
df_ids.to_csv('../data/list_ids.csv')

100%|██████████| 50/50 [00:14<00:00,  4.26it/s]
